In [1]:
import numpy as np
import xtools as xt
from typing import List, Union

from atm.flight.flight import Flight
from atm.flight.scenario import Scenario, ScenarioIterator
from atm.flight.category import Category
from atm.flight.operation import Operation, OperationMode
from atm.utility.random import poisson_interval


FL = List[Flight]


In [2]:
class ScenarioGenerator:

    def __init__(self, parameter: Union[xt.Config, dict]):
        if isinstance(parameter, dict):
            parameter = xt.Config(parameter)
        self._parameter = parameter
        self._interval: float = 1 / parameter.interval
        self._window: int = parameter.window
        self._mode: OperationMode = OperationMode.get(
            "mix" if not hasattr(parameter, "mode") else parameter.mode
        )
        self._category: Category = Category.standard(
            "recat" if not hasattr(parameter, "standard") else parameter.standard
        )
        if not hasattr(parameter, "seed"):
            seed = np.random.rand()
            parameter.set("seed", seed)
        self._rs = np.random.RandomState(parameter.seed)

    def __call__(self, num_generate: int) -> Scenario:
        # time window
        intervals = poisson_interval(self._interval, size=num_generate, rs=self._rs).astype(int)
        times_de = np.cumsum(intervals)
        times_to = np.ones_like(times_de) * self._window + times_de

        # mode
        if self._mode == OperationMode.M:
            operations = self._rs.choice(
                [o for o in Operation],
                num_generate, replace=True
            )
        else:
            operations = np.array([self._mode,] * num_generate)

        # category
        categories = self._rs.choice(
            [c for c in self._category],
            num_generate, replace=True
        )

        # summarize
        code_root = "VOL{:04.0f}"
        vols = [
            Flight(code_root.format(id_+1), tde, tto, ope, cat)
            for id_, (tde, tto, ope, cat) in enumerate(
                zip(times_de, times_to, operations, categories)
            )
        ]
        return Scenario(vols)

In [3]:
cf = xt.Config(dict(
    dt=10,
    num=30,
    scenario=dict(
        interval=60,
        window=300,
        mode="mix",
        standard="recat",
        seed=0,
    )
))


In [10]:
gen = ScenarioGenerator(cf.scenario)
vols = gen(cf.num)
vols.to_dataframe()

,code,ready,due,category,operation
0,VOL0001,35,335,D,D
1,VOL0002,55,355,C,A
2,VOL0003,85,385,D,D
3,VOL0004,121,421,A,A
4,VOL0005,172,472,D,D
5,VOL0006,198,498,F,A
6,VOL0007,247,547,E,A
7,VOL0008,253,553,B,A
8,VOL0009,255,555,C,A
9,VOL0010,312,612,E,A


In [11]:
gen = ScenarioGenerator(cf.scenario)
vols = gen(cf.num)
vols.to_dataframe()

,code,ready,due,category,operation
0,VOL0001,35,335,D,D
1,VOL0002,55,355,C,A
2,VOL0003,85,385,D,D
3,VOL0004,121,421,A,A
4,VOL0005,172,472,D,D
5,VOL0006,198,498,F,A
6,VOL0007,247,547,E,A
7,VOL0008,253,553,B,A
8,VOL0009,255,555,C,A
9,VOL0010,312,612,E,A
